## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [31]:
import mne
import numpy as np

file_path = "datasets/avi/single/sub1_combined-raw.fif"

epochs = mne.read_epochs(file_path, preload=True)

freq = 6.0  
threshold = 0.2  
fmin = freq - threshold / 2
fmax = freq + threshold / 2

epochs_filtered = epochs.copy().filter(l_freq=fmin, h_freq=fmax)
stimulated_frequencies = [6.0, 6.5, 7.0, 7.5, 8.2, 9.3,10]  
psd_values = []



for freq in stimulated_frequencies:
    # filtra os dados para a faixa de frequencia 
    test_filtered = epochs.copy().filter(l_freq=fmin, h_freq=fmax)
    
    # computar PSD
    psd = test_filtered.copy().get_data().mean(axis=0)
    psd_values.append(psd)

# passo 4: extrair recursos dos valores PSD
feature_vectors = []

for psd in psd_values:
    max_value = psd.max()
    average = psd.mean()
    median = np.median(psd)
    feature_vector = [max_value, average, median]  
    feature_vectors.append(feature_vector)

# passo 5: classificar os vetores de recursos
predictions = []
for feature_vector in feature_vectors:
    predicted_freq_index = np.argmax(feature_vector)
    print(feature_vector)
    predicted_freq = stimulated_frequencies[predicted_freq_index]
    predictions.append(predicted_freq)


print("vals previstos: {}".format(predictions))
print("frequencias: {}".format(stimulated_frequencies))


Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\datasets\avi\single\sub1_combined-raw.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   29998.05 ms
        0 CTF compensation matrices available
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Filter length: 845 samples (1.650 s)

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal

C:\Users\caio-\AppData\Local\Temp\ipykernel_27528\3555403522.py:8: RuntimeWarning: This filename (datasets/avi/single/sub1_combined-raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file_path, preload=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Filter length: 845 samples (1.650 s)

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Fi

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Filter length: 845 samples (1.650 s)

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Fi

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.

Setting up band-pass filter from 5.9 - 6.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.90 Hz)
- Upper passband edge: 6.10 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.10 Hz)
- Filter length: 845 samples (1.650 s)

[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.1557613146269565e-09]
[7.669229455803528e-07, -2.84296189360382e-10, -3.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:    0.0s finished
